# Task 2.3

## Thêm các thư viện cần thiết

In [1]:
import findspark
findspark.init()

In [2]:
# 1. Import các thư viện cần thiết
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql import functions as F
from pyspark.sql.types import BooleanType

# 2. Thiết lập đúng Python cho worker
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable


## Tạo session

In [3]:
spark=SparkSession.builder.master("local")\
    .appName("task2.3")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()

## Đọc dữ liệu file shapes.parquet

In [4]:
raw_df=spark.read.parquet('shapes.parquet')
raw_df.show(truncate=False)

+--------+--------------------------------------------+
|shape_id|vertices                                    |
+--------+--------------------------------------------+
|Shape_0 |[[35, 28], [43, 28], [43, 32], [35, 32]]    |
|Shape_1 |[[44, 16], [53, 16], [53, 20], [44, 20]]    |
|Shape_2 |[[67, 84], [76, 84], [76, 92], [67, 92]]    |
|Shape_3 |[[29, 37], [37, 37], [37, 45], [29, 45]]    |
|Shape_4 |[[51, 79], [54, 79], [54, 88], [51, 88]]    |
|Shape_5 |[[55, 33], [62, 33], [62, 39], [55, 39]]    |
|Shape_6 |[[17, 88], [21, 88], [21, 97], [17, 97]]    |
|Shape_7 |[[1, 2], [10, 2], [10, 7], [1, 7]]          |
|Shape_8 |[[65, 14], [69, 14], [69, 20], [65, 20]]    |
|Shape_9 |[[12, 48], [19, 48], [19, 51], [12, 51]]    |
|Shape_10|[[97, 20], [100, 20], [100, 26], [97, 26]]  |
|Shape_11|[[7, 38], [15, 38], [15, 45], [7, 45]]      |
|Shape_12|[[69, 65], [78, 65], [78, 68], [69, 68]]    |
|Shape_13|[[35, 8], [42, 8], [42, 18], [35, 18]]      |
|Shape_14|[[96, 35], [100, 35], [100, 38], [96, 

## In ra schema của DataFrame

In [5]:
raw_df.printSchema()

root
 |-- shape_id: string (nullable = true)
 |-- vertices: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: long (containsNull = true)



## Chuyển shape_id thành dạng số.

In [6]:
shape_df=raw_df.withColumn("shape_id", F.regexp_extract("shape_id", r"(\d+)", 1).cast("int"))
shape_df.show(truncate=False)

+--------+--------------------------------------------+
|shape_id|vertices                                    |
+--------+--------------------------------------------+
|0       |[[35, 28], [43, 28], [43, 32], [35, 32]]    |
|1       |[[44, 16], [53, 16], [53, 20], [44, 20]]    |
|2       |[[67, 84], [76, 84], [76, 92], [67, 92]]    |
|3       |[[29, 37], [37, 37], [37, 45], [29, 45]]    |
|4       |[[51, 79], [54, 79], [54, 88], [51, 88]]    |
|5       |[[55, 33], [62, 33], [62, 39], [55, 39]]    |
|6       |[[17, 88], [21, 88], [21, 97], [17, 97]]    |
|7       |[[1, 2], [10, 2], [10, 7], [1, 7]]          |
|8       |[[65, 14], [69, 14], [69, 20], [65, 20]]    |
|9       |[[12, 48], [19, 48], [19, 51], [12, 51]]    |
|10      |[[97, 20], [100, 20], [100, 26], [97, 26]]  |
|11      |[[7, 38], [15, 38], [15, 45], [7, 45]]      |
|12      |[[69, 65], [78, 65], [78, 68], [69, 68]]    |
|13      |[[35, 8], [42, 8], [42, 18], [35, 18]]      |
|14      |[[96, 35], [100, 35], [100, 38], [96, 

In [7]:
shape_df.printSchema()

root
 |-- shape_id: integer (nullable = true)
 |-- vertices: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: long (containsNull = true)



## Hàm kiểm tra có phải là hình chữ nhật hay không

In [8]:
def is_rectangle(vertices):
    A= vertices[0]
    B= vertices[1]
    C= vertices[2]
    D= vertices[3]
    
    AB = [B[0] - A[0], B[1] - A[1]]
    BC = [C[0] - B[0], C[1] - B[1]]
    CD = [D[0] - C[0], D[1] - C[1]]
    DA = [A[0] - D[0], A[1] - D[1]]

    dot_AB_BC= AB[0] * BC[0] + AB[1] * BC[1]
    dot_BC_CD= BC[0] * CD[0] + BC[1] * CD[1]
    dot_CD_DA= CD[0] * DA[0] + CD[1] * DA[1]
    dot_DA_AB= DA[0] * AB[0] + DA[1] * AB[1]

    if abs(dot_AB_BC) < 1e-6 and abs(dot_BC_CD) < 1e-6 and abs(dot_CD_DA) < 1e-6 and abs(dot_DA_AB) < 1e-6:
        return True
    else:
        return False


is_rectangle_udf = udf(is_rectangle, BooleanType())

## Kiểm tra cho các hình

In [9]:
shape_df.withColumn("is_rectangle", is_rectangle_udf("vertices")).select("is_rectangle").distinct().show(truncate=False)

+------------+
|is_rectangle|
+------------+
|true        |
+------------+



$\rightarrow$ Các hình đều là hình chữ nhật nếu ta xem 4 điểm này tạo thành tứ giác bao lồi

## Rã các điểm thành điểm x_min, x_max, y_min và y_max

In [10]:
shape_df=shape_df.withColumn("x_min",F.least(F.col("vertices")[0][0],F.col("vertices")[1][0],F.col("vertices")[2][0],F.col("vertices")[3][0]))\
    .withColumn("x_max",F.greatest(F.col("vertices")[0][0],F.col("vertices")[1][0],F.col("vertices")[2][0],F.col("vertices")[3][0]))\
    .withColumn("y_min",F.least(F.col("vertices")[0][1],F.col("vertices")[1][1],F.col("vertices")[2][1],F.col("vertices")[3][1]))\
    .withColumn("y_max",F.greatest(F.col("vertices")[0][1],F.col("vertices")[1][1],F.col("vertices")[2][1],F.col("vertices")[3][1]))\
    .select("shape_id","x_min","x_max","y_min","y_max")

shape_df.show(truncate=False)
    

+--------+-----+-----+-----+-----+
|shape_id|x_min|x_max|y_min|y_max|
+--------+-----+-----+-----+-----+
|0       |35   |43   |28   |32   |
|1       |44   |53   |16   |20   |
|2       |67   |76   |84   |92   |
|3       |29   |37   |37   |45   |
|4       |51   |54   |79   |88   |
|5       |55   |62   |33   |39   |
|6       |17   |21   |88   |97   |
|7       |1    |10   |2    |7    |
|8       |65   |69   |14   |20   |
|9       |12   |19   |48   |51   |
|10      |97   |100  |20   |26   |
|11      |7    |15   |38   |45   |
|12      |69   |78   |65   |68   |
|13      |35   |42   |8    |18   |
|14      |96   |100  |35   |38   |
|15      |74   |81   |62   |69   |
|16      |75   |80   |97   |102  |
|17      |12   |19   |4    |12   |
|18      |54   |57   |17   |20   |
|19      |100  |106  |66   |70   |
+--------+-----+-----+-----+-----+
only showing top 20 rows



## Ghép shape_df với shape_df sao cho shape_2 > shape_1

In [11]:
df1 = shape_df.alias("a")
df2 = shape_df.alias("b")

# Join để tạo các cặp shape_id khác nhau sao cho shape_id_a < shape_id_b
pairs_df = df1.join(df2, F.col("a.shape_id") < F.col("b.shape_id"),'inner').selectExpr("a.shape_id as shape_1", "a.x_min as x_min_1", "a.x_max as x_max_1", "a.y_min as y_min_1", "a.y_max as y_max_1",
                    "b.shape_id as shape_2", "b.x_min as x_min_2", "b.x_max as x_max_2", "b.y_min as y_min_2", "b.y_max as y_max_2")
pairs_df.show(truncate=False)



+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|shape_1|x_min_1|x_max_1|y_min_1|y_max_1|shape_2|x_min_2|x_max_2|y_min_2|y_max_2|
+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|0      |35     |43     |28     |32     |1      |44     |53     |16     |20     |
|0      |35     |43     |28     |32     |2      |67     |76     |84     |92     |
|0      |35     |43     |28     |32     |3      |29     |37     |37     |45     |
|0      |35     |43     |28     |32     |4      |51     |54     |79     |88     |
|0      |35     |43     |28     |32     |5      |55     |62     |33     |39     |
|0      |35     |43     |28     |32     |6      |17     |21     |88     |97     |
|0      |35     |43     |28     |32     |7      |1      |10     |2      |7      |
|0      |35     |43     |28     |32     |8      |65     |69     |14     |20     |
|0      |35     |43     |28     |32     |9      |12     |19     |48     |51     |
|0      |35     

## Hàm kiểm tra overlap

In [12]:
def is_overlap(x_min_1, x_max_1, y_min_1, y_max_1, x_min_2, x_max_2, y_min_2, y_max_2):
    if x_min_1 >= x_max_2 or x_min_2 >= x_max_1:
        return False
    if y_min_1 >= y_max_2 or y_min_2 >= y_max_1:
        return False
    return True

is_overlap_udf = udf(is_overlap, BooleanType())

## Kiểm tra overlap

In [13]:
check_overlap_df = pairs_df.withColumn(
    "is_overlap",
    is_overlap_udf("x_min_1", "x_max_1", "y_min_1", "y_max_1",
                  "x_min_2", "x_max_2", "y_min_2", "y_max_2")
)
check_overlap_df.show(truncate=False)

+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+----------+
|shape_1|x_min_1|x_max_1|y_min_1|y_max_1|shape_2|x_min_2|x_max_2|y_min_2|y_max_2|is_overlap|
+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+----------+
|0      |35     |43     |28     |32     |1      |44     |53     |16     |20     |false     |
|0      |35     |43     |28     |32     |2      |67     |76     |84     |92     |false     |
|0      |35     |43     |28     |32     |3      |29     |37     |37     |45     |false     |
|0      |35     |43     |28     |32     |4      |51     |54     |79     |88     |false     |
|0      |35     |43     |28     |32     |5      |55     |62     |33     |39     |false     |
|0      |35     |43     |28     |32     |6      |17     |21     |88     |97     |false     |
|0      |35     |43     |28     |32     |7      |1      |10     |2      |7      |false     |
|0      |35     |43     |28     |32     |8      |65     |69     |14   

## Lọc các trường hợp có giá trị bằng true

In [14]:
overlap_df=check_overlap_df.filter(F.col("is_overlap") == 'true').select("shape_1","shape_2").orderBy(F.asc("shape_1"),F.asc("shape_2"))
overlap_df.show(truncate=False)

+-------+-------+
|shape_1|shape_2|
+-------+-------+
|0      |41     |
|0      |143    |
|0      |198    |
|0      |430    |
|0      |473    |
|0      |490    |
|0      |496    |
|0      |605    |
|0      |637    |
|0      |729    |
|0      |818    |
|1      |118    |
|1      |160    |
|1      |186    |
|1      |220    |
|1      |247    |
|1      |256    |
|1      |377    |
|1      |444    |
|1      |809    |
+-------+-------+
only showing top 20 rows



## Xuất file csv

In [15]:
overlap_df.coalesce(1).write.csv('output', header=True, mode='overwrite')